In [1]:
import sqlite3 
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()

In [2]:
import pandas as pd

In [3]:
c.execute("""SELECT * FROM Matches;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head(10)

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A
5,6,D2,2009,2010-04-03,Union Berlin,Karlsruhe,1,1,D
6,7,D2,2009,2009-08-14,Paderborn,Karlsruhe,2,0,H
7,8,D2,2009,2010-03-08,Bielefeld,Karlsruhe,0,1,A
8,9,D2,2009,2009-09-26,Kaiserslautern,Karlsruhe,2,0,H
9,10,D2,2009,2009-11-21,Hansa Rostock,Karlsruhe,2,1,H


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24625 entries, 0 to 24624
Data columns (total 9 columns):
Match_ID    24625 non-null int64
Div         24625 non-null object
Season      24625 non-null int64
Date        24625 non-null object
HomeTeam    24625 non-null object
AwayTeam    24625 non-null object
FTHG        24625 non-null int64
FTAG        24625 non-null int64
FTR         24625 non-null object
dtypes: int64(4), object(5)
memory usage: 1.7+ MB


In [5]:
df_2011 = df.loc[df['Season']==2011]

In [6]:
df_2011.head(10)

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
1091,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1092,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
1093,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
1094,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
1095,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D
1096,1097,D1,2011,2012-01-20,M'gladbach,Bayern Munich,3,1,H
1097,1098,D1,2011,2012-02-04,Hamburg,Bayern Munich,1,1,D
1098,1099,D1,2011,2012-04-21,Werder Bremen,Bayern Munich,1,2,A
1099,1100,D1,2011,2011-09-18,Schalke 04,Bayern Munich,0,2,A
1100,1101,D1,2011,2011-10-23,Hannover,Bayern Munich,2,1,H


In [7]:
df_2011.shape

(992, 9)

In [12]:
c.execute("""SELECT Unique_Team_ID, Div, HomeTeam, SUM(FTHG) as hgoals FROM Matches
                JOIN Unique_Teams ON TeamName=HomeTeam
                WHERE Season = 2011
                GROUP BY HomeTeam
                ORDER BY Unique_Team_ID;""")
hgoals_df = pd.DataFrame(c.fetchall())
hgoals_df.columns = [x[0] for x in c.description]
hgoals_df.head()

,Unique_Team_ID,Div,HomeTeam,hgoals
0,1,D1,Bayern Munich,49
1,2,D1,Dortmund,44
2,3,D1,Leverkusen,28
3,5,D1,Schalke 04,47
4,6,D1,M'gladbach,29


In [38]:
#hgoals_df['HomeTeam'].unique

In [13]:
c.execute("""SELECT Unique_Team_ID, Div, AwayTeam, SUM(FTAG) as agoals FROM Matches
                JOIN Unique_Teams ON TeamName=AwayTeam
                WHERE Season = 2011
                GROUP BY AwayTeam
                ORDER BY AwayTeam;""")
agoals_df = pd.DataFrame(c.fetchall())
agoals_df.columns = [x[0] for x in c.description]
agoals_df.head()

,Unique_Team_ID,Div,AwayTeam,agoals
0,34,D2,Aachen,15
1,81,E0,Arsenal,35
2,82,E0,Aston Villa,17
3,15,D1,Augsburg,16
4,1,D1,Bayern Munich,28


In [39]:
#agoals_df['AwayTeam'].unique

In [14]:
#team home+away goals
goals_df = pd.merge(hgoals_df, agoals_df, on='Unique_Team_ID')

In [15]:
goals_df['total_goals'] = goals_df['hgoals'] + goals_df['agoals']

In [16]:
goals_df = goals_df.drop(['Div_y','AwayTeam'], axis=1)

In [17]:
goals_df.head()

,Unique_Team_ID,Div_x,HomeTeam,hgoals,agoals,total_goals
0,1,D1,Bayern Munich,49,28,77
1,2,D1,Dortmund,44,36,80
2,3,D1,Leverkusen,28,24,52
3,5,D1,Schalke 04,47,27,74
4,6,D1,M'gladbach,29,20,49


In [18]:
#home wins
c.execute("""SELECT Unique_Team_ID, Div, HomeTeam, COUNT(FTR) as hwins FROM Matches
                JOIN Unique_Teams ON TeamName=HomeTeam
                WHERE Season = 2011 AND FTR is 'H'
                GROUP BY HomeTeam
                ORDER BY Unique_Team_ID;""")
hwins_df = pd.DataFrame(c.fetchall())
hwins_df.columns = [x[0] for x in c.description]
hwins_df.head()

,Unique_Team_ID,Div,HomeTeam,hwins
0,1,D1,Bayern Munich,14
1,2,D1,Dortmund,14
2,3,D1,Leverkusen,8
3,5,D1,Schalke 04,13
4,6,D1,M'gladbach,9


In [19]:
hwins_df.shape

(56, 4)

In [20]:
#away wins
c.execute("""SELECT Unique_Team_ID, Div, AwayTeam, COUNT(FTR) as awins FROM Matches
                JOIN Unique_Teams ON TeamName=AwayTeam
                WHERE Season = 2011 AND FTR is 'A'
                GROUP BY AwayTeam
                ORDER BY Unique_Team_ID;""")
awins_df = pd.DataFrame(c.fetchall())
awins_df.columns = [x[0] for x in c.description]
awins_df.head()

,Unique_Team_ID,Div,AwayTeam,awins
0,1,D1,Bayern Munich,9
1,2,D1,Dortmund,11
2,3,D1,Leverkusen,7
3,5,D1,Schalke 04,7
4,6,D1,M'gladbach,8


In [21]:
awins_df.shape

(56, 4)

In [22]:
#home losses
c.execute("""SELECT Unique_Team_ID, Div, HomeTeam, COUNT(FTR) as hloss FROM Matches
                JOIN Unique_Teams ON TeamName=HomeTeam
                WHERE Season = 2011 AND FTR is 'A'
                GROUP BY HomeTeam
                ORDER BY Unique_Team_ID;""")
hloss_df = pd.DataFrame(c.fetchall())
hloss_df.columns = [x[0] for x in c.description]
hloss_df.head()

,Unique_Team_ID,Div,HomeTeam,hloss
0,1,D1,Bayern Munich,2
1,2,D1,Dortmund,1
2,3,D1,Leverkusen,5
3,5,D1,Schalke 04,3
4,6,D1,M'gladbach,1


In [23]:
hloss_df.shape

(54, 4)

In [24]:
#away losses
c.execute("""SELECT Unique_Team_ID, Div, AwayTeam, COUNT(FTR) as aloss FROM Matches
                JOIN Unique_Teams ON TeamName=AwayTeam
                WHERE Season = 2011 AND FTR is 'H'
                GROUP BY AwayTeam
                ORDER BY Unique_Team_ID;""")
aloss_df = pd.DataFrame(c.fetchall())
aloss_df.columns = [x[0] for x in c.description]
aloss_df.head()

,Unique_Team_ID,Div,AwayTeam,aloss
0,1,D1,Bayern Munich,5
1,2,D1,Dortmund,2
2,3,D1,Leverkusen,5
3,5,D1,Schalke 04,7
4,6,D1,M'gladbach,7


In [25]:
#home draws
c.execute("""SELECT Unique_Team_ID, Div, HomeTeam, COUNT(FTR) as hdraw FROM Matches
                JOIN Unique_Teams ON TeamName=HomeTeam
                WHERE Season = 2011 AND FTR is 'D'
                GROUP BY HomeTeam
                ORDER BY Unique_Team_ID;""")
hdraw_df = pd.DataFrame(c.fetchall())
hdraw_df.columns = [x[0] for x in c.description]
hdraw_df.head()

,Unique_Team_ID,Div,HomeTeam,hdraw
0,1,D1,Bayern Munich,1
1,2,D1,Dortmund,2
2,3,D1,Leverkusen,4
3,5,D1,Schalke 04,1
4,6,D1,M'gladbach,7


In [26]:
#away draws
c.execute("""SELECT Unique_Team_ID, Div, AwayTeam, COUNT(FTR) as adraw FROM Matches
                JOIN Unique_Teams ON TeamName=AwayTeam
                WHERE Season = 2011 AND FTR is 'D'
                GROUP BY AwayTeam
                ORDER BY Unique_Team_ID;""")
adraw_df = pd.DataFrame(c.fetchall())
adraw_df.columns = [x[0] for x in c.description]
adraw_df.head()

,Unique_Team_ID,Div,AwayTeam,adraw
0,1,D1,Bayern Munich,3
1,2,D1,Dortmund,4
2,3,D1,Leverkusen,5
3,5,D1,Schalke 04,3
4,6,D1,M'gladbach,2


In [74]:
#combine home and away wins

In [27]:
#merge wins
record_df = pd.merge(hwins_df, awins_df[['Unique_Team_ID', 'awins']], on='Unique_Team_ID')

In [28]:
#merge losses
record_df = pd.merge(record_df, hloss_df[['Unique_Team_ID', 'hloss']], on='Unique_Team_ID', how='left')

In [29]:
record_df = pd.merge(record_df, aloss_df[['Unique_Team_ID', 'aloss']], on='Unique_Team_ID', how='left')

In [30]:
#merge draws
record_df = pd.merge(record_df, hdraw_df[['Unique_Team_ID', 'hdraw']], on='Unique_Team_ID', how='left')

In [31]:
record_df = pd.merge(record_df, adraw_df[['Unique_Team_ID', 'adraw']], on='Unique_Team_ID', how='left')

In [32]:
record_df.head()

,Unique_Team_ID,Div,HomeTeam,hwins,awins,hloss,aloss,hdraw,adraw
0,1,D1,Bayern Munich,14,9,2.0,5,1,3
1,2,D1,Dortmund,14,11,1.0,2,2,4
2,3,D1,Leverkusen,8,7,5.0,5,4,5
3,5,D1,Schalke 04,13,7,3.0,7,1,3
4,6,D1,M'gladbach,9,8,1.0,7,7,2


In [33]:
record_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56 entries, 0 to 55
Data columns (total 9 columns):
Unique_Team_ID    56 non-null int64
Div               56 non-null object
HomeTeam          56 non-null object
hwins             56 non-null int64
awins             56 non-null int64
hloss             54 non-null float64
aloss             56 non-null int64
hdraw             56 non-null int64
adraw             56 non-null int64
dtypes: float64(1), int64(6), object(2)
memory usage: 4.4+ KB


In [34]:
record_df['hloss'] = record_df['hloss'].apply(lambda x: 0 if pd.isnull(x) else int(x))

In [35]:
record_df['total_matches'] = (record_df['hwins'] + record_df['awins'] + record_df['hloss'] + record_df['aloss'] + 
                             record_df['hdraw'] + record_df['adraw'])

In [36]:
record_df.head()

,Unique_Team_ID,Div,HomeTeam,hwins,awins,hloss,aloss,hdraw,adraw,total_matches
0,1,D1,Bayern Munich,14,9,2,5,1,3,34
1,2,D1,Dortmund,14,11,1,2,2,4,34
2,3,D1,Leverkusen,8,7,5,5,4,5,34
3,5,D1,Schalke 04,13,7,3,7,1,3,34
4,6,D1,M'gladbach,9,8,1,7,7,2,34


In [37]:
df_2011['Div'].value_counts()

E0    380
D1    306
D2    306
Name: Div, dtype: int64

In [38]:
record_df['tot_wins'] = record_df['hwins']+record_df['awins']
record_df['tot_draws'] = record_df['hdraw']+record_df['adraw']
record_df['tot_losses'] = record_df['hloss']+record_df['aloss']

In [39]:
record_df['TeamName'] = record_df['HomeTeam']

In [40]:
record_df.head()

,Unique_Team_ID,Div,HomeTeam,hwins,awins,hloss,aloss,hdraw,adraw,total_matches,tot_wins,tot_draws,tot_losses,TeamName
0,1,D1,Bayern Munich,14,9,2,5,1,3,34,23,4,7,Bayern Munich
1,2,D1,Dortmund,14,11,1,2,2,4,34,25,6,3,Dortmund
2,3,D1,Leverkusen,8,7,5,5,4,5,34,15,9,10,Leverkusen
3,5,D1,Schalke 04,13,7,3,7,1,3,34,20,4,10,Schalke 04
4,6,D1,M'gladbach,9,8,1,7,7,2,34,17,9,8,M'gladbach


In [41]:
record_df = record_df[['Unique_Team_ID', 'Div','TeamName', 'tot_wins', 'tot_draws', 'tot_losses', 'total_matches']]

### Consolidated Records Table

In [42]:
record_df.head(10)

,Unique_Team_ID,Div,TeamName,tot_wins,tot_draws,tot_losses,total_matches
0,1,D1,Bayern Munich,23,4,7,34
1,2,D1,Dortmund,25,6,3,34
2,3,D1,Leverkusen,15,9,10,34
3,5,D1,Schalke 04,20,4,10,34
4,6,D1,M'gladbach,17,9,8,34
5,7,D1,Wolfsburg,13,5,16,34
6,8,D1,FC Koln,8,6,20,34
7,9,D1,Hoffenheim,10,11,13,34
8,10,D1,Hertha,7,10,17,34
9,11,D1,Mainz,9,12,13,34


In [43]:
df_2011.loc[df_2011['Div'] == 'E0'].head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
22345,44495,E0,2011,2011-08-13,Blackburn,Wolves,1,2,A
22346,44496,E0,2011,2011-08-13,Fulham,Aston Villa,0,0,D
22347,44497,E0,2011,2011-08-13,Liverpool,Sunderland,1,1,D
22348,44498,E0,2011,2011-08-13,Newcastle,Arsenal,0,0,D
22349,44499,E0,2011,2011-08-13,QPR,Bolton,0,4,A


In [44]:
c.execute("""SELECT * FROM Teams_in_Matches;""")
match_df = pd.DataFrame(c.fetchall())
match_df.columns = [x[0] for x in c.description]
match_df.head()

,Match_ID,Unique_Team_ID
0,1,26
1,1,46
2,2,26
3,2,42
4,3,26


In [45]:
c.execute("""SELECT * FROM Unique_Teams;""")
team_df = pd.DataFrame(c.fetchall())
team_df.columns = [x[0] for x in c.description]
team_df.head()

,TeamName,Unique_Team_ID
0,Bayern Munich,1
1,Dortmund,2
2,Leverkusen,3
3,RB Leipzig,4
4,Schalke 04,5


In [46]:
df_2011.head(10)

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
1091,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1092,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
1093,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
1094,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
1095,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D
1096,1097,D1,2011,2012-01-20,M'gladbach,Bayern Munich,3,1,H
1097,1098,D1,2011,2012-02-04,Hamburg,Bayern Munich,1,1,D
1098,1099,D1,2011,2012-04-21,Werder Bremen,Bayern Munich,1,2,A
1099,1100,D1,2011,2011-09-18,Schalke 04,Bayern Munich,0,2,A
1100,1101,D1,2011,2011-10-23,Hannover,Bayern Munich,2,1,H


In [48]:
goals_df2 = goals_df[['Unique_Team_ID', 'total_goals']]

In [51]:
record_df = pd.merge(record_df, goals_df2, on='Unique_Team_ID')

In [53]:
record_df.head(10)

,Unique_Team_ID,Div,TeamName,tot_wins,tot_draws,tot_losses,total_matches,total_goals
0,1,D1,Bayern Munich,23,4,7,34,77
1,2,D1,Dortmund,25,6,3,34,80
2,3,D1,Leverkusen,15,9,10,34,52
3,5,D1,Schalke 04,20,4,10,34,74
4,6,D1,M'gladbach,17,9,8,34,49
5,7,D1,Wolfsburg,13,5,16,34,47
6,8,D1,FC Koln,8,6,20,34,39
7,9,D1,Hoffenheim,10,11,13,34,41
8,10,D1,Hertha,7,10,17,34,38
9,11,D1,Mainz,9,12,13,34,47


In [55]:
record_df.loc[record_df['Div']=='E0'].head(10)

,Unique_Team_ID,Div,TeamName,tot_wins,tot_draws,tot_losses,total_matches,total_goals
36,81,E0,Arsenal,21,7,10,38,74
37,82,E0,Aston Villa,7,17,14,38,37
38,83,E0,Chelsea,18,10,10,38,65
39,84,E0,Liverpool,14,10,14,38,47
40,85,E0,Man City,28,5,5,38,93
41,86,E0,Newcastle,19,8,11,38,56
42,91,E0,Norwich,12,11,15,38,52
43,92,E0,Tottenham,20,9,9,38,66
44,93,E0,Everton,15,11,12,38,50
45,97,E0,Blackburn,8,7,23,38,48
